In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16858,2022-05-06T17:00:00,ITA,19,Sicilia,38.115697,13.362357,745,40,785,111999,...,DECEDUTI: N. 2 IL 05/05/2022 - N. 4 IL 04/05/2...,0.0,NaN,Sul numero complessivo dei casi confermati com...,503313.0,629588.0,4650040.0,8062741.0,ITG,ITG1
16859,2022-05-06T17:00:00,ITA,9,Toscana,43.769231,11.255889,518,24,542,44112,...,NaN,6.0,NaN,NaN,624758.0,487034.0,6748078.0,7119071.0,ITI,ITI1
16860,2022-05-06T17:00:00,ITA,10,Umbria,43.106758,12.388247,214,5,219,10985,...,- Si fa presente che 2 dei ricoveri non UTI ap...,0.0,NaN,NaN,138629.0,134422.0,1632945.0,2598457.0,ITI,ITI2
16861,2022-05-06T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,28,0,28,1393,...,NaN,0.0,NaN,NaN,13101.0,22382.0,139060.0,372177.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16858,2022-05-06T17:00:00,19,Sicilia,745,40,785,111999,112784,-1413,3000,...,1132901,12712781,7135228.0,0.0,503313.0,629588.0,4650040.0,8062741.0,ITG,ITG1
16859,2022-05-06T17:00:00,9,Toscana,518,24,542,44112,44654,-1121,2234,...,1111792,13867149,4750640.0,6.0,624758.0,487034.0,6748078.0,7119071.0,ITI,ITI1
16860,2022-05-06T17:00:00,10,Umbria,214,5,219,10985,11204,121,926,...,273051,4231402,719964.0,0.0,138629.0,134422.0,1632945.0,2598457.0,ITI,ITI2
16861,2022-05-06T17:00:00,2,Valle d'Aosta,28,0,28,1393,1421,-21,86,...,35483,511237,133618.0,0.0,13101.0,22382.0,139060.0,372177.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-05-06', '2022-05-05')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-05-06T17:00:00,1,Piemonte,705,18,723,57373,58096,-3684,2513,...,1159135,17070990,3950047.0,1.0,499890.0,659245.0,4909595.0,12161395.0,ITC,ITC1
1,2022-05-06T17:00:00,2,Valle d'Aosta,28,0,28,1393,1421,-21,86,...,35483,511237,133618.0,0.0,13101.0,22382.0,139060.0,372177.0,ITC,ITC2
2,2022-05-06T17:00:00,3,Lombardia,1149,36,1185,145555,146740,-1114,5747,...,2800489,36949477,8225973.0,0.0,1405057.0,1395432.0,15843110.0,21106367.0,ITC,ITC4
3,2022-05-06T17:00:00,5,Veneto,534,20,554,61787,62341,-753,4464,...,1694104,29308619,4790920.0,7.0,800369.0,893735.0,9901055.0,19407564.0,ITH,ITH3
4,2022-05-06T17:00:00,6,Friuli Venezia Giulia,160,7,167,23670,23837,-51,763,...,367760,6393661,1132454.0,1.0,198807.0,168953.0,3314838.0,3078823.0,ITH,ITH4
5,2022-05-06T17:00:00,7,Liguria,275,15,290,14670,14960,-203,1134,...,433233,5416982,1311353.0,0.0,232925.0,200308.0,2435322.0,2981660.0,ITC,ITC3
6,2022-05-06T17:00:00,8,Emilia-Romagna,1332,32,1364,49653,51017,-2301,3595,...,1431893,15841757,2733473.0,3.0,927681.0,504212.0,9112256.0,6729501.0,ITH,ITH5
7,2022-05-06T17:00:00,9,Toscana,518,24,542,44112,44654,-1121,2234,...,1111792,13867149,4750640.0,6.0,624758.0,487034.0,6748078.0,7119071.0,ITI,ITI1
8,2022-05-06T17:00:00,10,Umbria,214,5,219,10985,11204,121,926,...,273051,4231402,719964.0,0.0,138629.0,134422.0,1632945.0,2598457.0,ITI,ITI2
9,2022-05-06T17:00:00,11,Marche,150,5,155,6425,6580,-220,1611,...,452799,3040952,2039634.0,1.0,214303.0,238496.0,1971576.0,1069376.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-05-05T17:00:00,1,Piemonte,735,17,752,61028,61780,586,2824,...,1156622,17048397,3947192.0,2.0,499515.0,657107.0,4907191.0,12141206.0,ITC,ITC1
1,2022-05-05T17:00:00,2,Valle d'Aosta,27,0,27,1415,1442,-13,80,...,35397,510868,133572.0,0.0,13110.0,22287.0,139010.0,371858.0,ITC,ITC2
2,2022-05-05T17:00:00,3,Lombardia,1179,37,1216,146638,147854,-6672,6362,...,2794742,36908582,8221368.0,2.0,1404149.0,1390593.0,15834547.0,21074035.0,ITC,ITC4
3,2022-05-05T17:00:00,5,Veneto,565,17,582,62512,63094,-2190,5344,...,1689640,29266911,4787981.0,1.0,800596.0,889044.0,9892329.0,19374582.0,ITH,ITH3
4,2022-05-05T17:00:00,6,Friuli Venezia Giulia,171,9,180,23708,23888,-198,858,...,367001,6386160,1131909.0,4.0,198611.0,168390.0,3311100.0,3075060.0,ITH,ITH4
5,2022-05-05T17:00:00,7,Liguria,271,15,286,14877,15163,-147,1252,...,432099,5404715,1309821.0,0.0,232678.0,199421.0,2433718.0,2970997.0,ITC,ITC3
6,2022-05-05T17:00:00,8,Emilia-Romagna,1330,34,1364,51954,53318,-929,4041,...,1428299,15822868,2732222.0,5.0,926634.0,501665.0,9102201.0,6720667.0,ITH,ITH5
7,2022-05-05T17:00:00,9,Toscana,542,25,567,45208,45775,-472,2712,...,1109558,13851582,4747435.0,4.0,624224.0,485334.0,6745536.0,7106046.0,ITI,ITI1
8,2022-05-05T17:00:00,10,Umbria,216,6,222,10861,11083,-74,889,...,272125,4227166,719394.0,1.0,138446.0,133679.0,1632075.0,2595091.0,ITI,ITI2
9,2022-05-05T17:00:00,11,Marche,161,4,165,6635,6800,106,1469,...,451188,3036822,2035504.0,0.0,214179.0,237009.0,1971153.0,1065669.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-05-06T17:00:00,1,Piemonte,705,18,723,57373,58096,-3684,2513,...,17070990,3950047.0,1.0,499890.0,659245.0,4909595.0,12161395.0,ITC,ITC1,4.341375e+06
1,2022-05-06T17:00:00,2,Valle d'Aosta,28,0,28,1393,1421,-21,86,...,511237,133618.0,0.0,13101.0,22382.0,139060.0,372177.0,ITC,ITC2,1.255010e+05
2,2022-05-06T17:00:00,3,Lombardia,1149,36,1185,145555,146740,-1114,5747,...,36949477,8225973.0,0.0,1405057.0,1395432.0,15843110.0,21106367.0,ITC,ITC4,1.010397e+07
3,2022-05-06T17:00:00,5,Veneto,534,20,554,61787,62341,-753,4464,...,29308619,4790920.0,7.0,800369.0,893735.0,9901055.0,19407564.0,ITH,ITH3,4.907704e+06
4,2022-05-06T17:00:00,6,Friuli Venezia Giulia,160,7,167,23670,23837,-51,763,...,6393661,1132454.0,1.0,198807.0,168953.0,3314838.0,3078823.0,ITH,ITH4,1.211357e+06
5,2022-05-06T17:00:00,7,Liguria,275,15,290,14670,14960,-203,1134,...,5416982,1311353.0,0.0,232925.0,200308.0,2435322.0,2981660.0,ITC,ITC3,1.543127e+06
6,2022-05-06T17:00:00,8,Emilia-Romagna,1332,32,1364,49653,51017,-2301,3595,...,15841757,2733473.0,3.0,927681.0,504212.0,9112256.0,6729501.0,ITH,ITH5,4.467118e+06
7,2022-05-06T17:00:00,9,Toscana,518,24,542,44112,44654,-1121,2234,...,13867149,4750640.0,6.0,624758.0,487034.0,6748078.0,7119071.0,ITI,ITI1,3.722729e+06
8,2022-05-06T17:00:00,10,Umbria,214,5,219,10985,11204,121,926,...,4231402,719964.0,0.0,138629.0,134422.0,1632945.0,2598457.0,ITI,ITI2,8.802850e+05
9,2022-05-06T17:00:00,11,Marche,150,5,155,6425,6580,-220,1611,...,3040952,2039634.0,1.0,214303.0,238496.0,1971576.0,1069376.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-05-06T17:00:00,1,Piemonte,705,18,723,57373,58096,-3684,2513,...,12161395.0,ITC,ITC1,4.341375e+06,Piemonte,705,6794,18,628,99
1,2022-05-06T17:00:00,2,Valle d'Aosta,28,0,28,1393,1421,-21,86,...,372177.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,28,163,0,34,0
2,2022-05-06T17:00:00,3,Lombardia,1149,36,1185,145555,146740,-1114,5747,...,21106367.0,ITC,ITC4,1.010397e+07,Lombardia,1149,10457,36,1810,0
3,2022-05-06T17:00:00,5,Veneto,534,20,554,61787,62341,-753,4464,...,19407564.0,ITH,ITH3,4.907704e+06,Veneto,534,6000,20,1000,0
4,2022-05-06T17:00:00,6,Friuli Venezia Giulia,160,7,167,23670,23837,-51,763,...,3078823.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,160,1277,7,175,0
5,2022-05-06T17:00:00,7,Liguria,275,15,290,14670,14960,-203,1134,...,2981660.0,ITC,ITC3,1.543127e+06,Liguria,275,1643,15,218,12
6,2022-05-06T17:00:00,8,Emilia-Romagna,1332,32,1364,49653,51017,-2301,3595,...,6729501.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1332,9001,32,889,0
7,2022-05-06T17:00:00,9,Toscana,518,24,542,44112,44654,-1121,2234,...,7119071.0,ITI,ITI1,3.722729e+06,Toscana,518,5033,24,570,10
8,2022-05-06T17:00:00,10,Umbria,214,5,219,10985,11204,121,926,...,2598457.0,ITI,ITI2,8.802850e+05,Umbria,214,662,5,86,41
9,2022-05-06T17:00:00,11,Marche,150,5,155,6425,6580,-220,1611,...,1069376.0,ITI,ITI3,1.518400e+06,Marche,150,1006,5,232,52


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-05-06T17:00:00,3,Lombardia,1149,36,1185,145555,146740,-1114,5747,...,1.99,-1083,-615,40895,4605.0,29,6832,14.05,0.05688,27.72
1,2022-05-06T17:00:00,15,Campania,694,40,734,160042,160776,-3121,5009,...,7.05,-3116,-103,29204,3395.0,4,8126,17.15,0.08657,28.25
2,2022-05-06T17:00:00,5,Veneto,534,20,554,61787,62341,-753,4464,...,2.00,-725,-880,41708,2939.0,8,5209,10.70,0.09096,34.52
3,2022-05-06T17:00:00,12,Lazio,1065,58,1123,148482,149605,-1579,3807,...,6.15,-1527,-144,32040,3734.0,8,5378,11.88,0.06490,25.70
4,2022-05-06T17:00:00,8,Emilia-Romagna,1332,32,1364,49653,51017,-2301,3595,...,3.60,-2301,-446,18889,1251.0,11,5884,19.03,0.08048,32.05
5,2022-05-06T17:00:00,16,Puglia,525,26,551,95557,96108,-997,3109,...,5.17,-1002,-799,19491,3550.0,6,4100,15.95,0.07756,27.10
6,2022-05-06T17:00:00,19,Sicilia,745,40,785,111999,112784,-1413,3000,...,4.90,-1401,-263,20359,20096.0,11,4829,14.74,0.06038,22.80
7,2022-05-06T17:00:00,1,Piemonte,705,18,723,57373,58096,-3684,2513,...,2.87,-3655,-311,22593,2855.0,5,6192,11.12,0.05788,26.70
8,2022-05-06T17:00:00,9,Toscana,518,24,542,44112,44654,-1121,2234,...,4.21,-1096,-478,15567,3205.0,11,3344,14.35,0.06001,29.86
9,2022-05-06T17:00:00,13,Abruzzo,309,9,318,55795,56113,-229,1657,...,4.97,-222,-36,9872,735.0,6,1875,16.78,0.12690,29.36


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Campania,40,527,567,7.05
1,Liguria,15,203,218,6.88
2,Calabria,12,182,194,6.19
3,Lazio,58,885,943,6.15
4,Umbria,5,81,86,5.81
5,Puglia,26,477,503,5.17
6,Molise,2,37,39,5.13
7,Abruzzo,9,172,181,4.97
8,Sicilia,40,777,817,4.90
9,Sardegna,9,195,204,4.41


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))